# 02805 Social graphs and interactions, Fall 20205 - Assignment 1

In [9]:
import networkx as nx 
import matplotlib.pyplot as plt
import numpy as np
import pickle, requests, os, re

### Assignment 1.1: Exploring WS and BA models

#### Did you really read the text?

- Q: What's the problem with random networks as a model for real-world networks according to the argument in section 3.5?

A: XXX

- Q: List the four regimes that characterize random networks as a function of $<k>$.

A: XXX

- Q: According to the book, why is it a problem for random networks (in terms of being a model for real-world networks) that the degree-dependent clustering $C(k)$ decreases as a function of $k$ in real-world networks?

A: XXX

#### WS Edition

- Q: First, let's use networkx to play around with WS graphs. Use nx.watts_strogatz_graph to generate 3 graphs with 500 nodes each, average degree = 4, and rewiring probablity $p=0, 0.1,$ and $1$. Calculate the average shortest path length $<d>$ for each one.

- Q: Describe what happens to the network when $p=1$.

- Q: Generate a lot of networks with different values of $p$. You will notice that paths are short when $p$ is close to one and they are long when $p=0$. What's the value of $p$ for which the average shortest path length get close to the short paths we find in a fully randomized network.

- Q: Let's investigate this behavior in detail. Generate 50 networks with $N = 500, <k> = 4$, for each of $p \in \left\{ 0, 0.01, 0.03, 0.05, 0.1, 0.2\right\}$. Calculate the average of $<d>$ as well as the standard deviation over the 50 networks, to create a plot that shows how the path length decreases very quickly with only a little fraction of re-wiring. Use the standard deviation to add errorbars to the plot. My version of the plot is below (since a picture's worth 1000 words).

- Q: Imagine that you put this plot in an assignment. Write a figure caption that explains to the reader what the plot shows (which variables, etc) and what's interesting about it.

#### BA Edition

- Q: What are the three slope dependent regimes of complex networks with power-law degree distributions? Briefly describe each one. (You will have to skim chp 4.7 to answer this one).

A: XXX

- Q: What are the three regimes we find in non-linear preferential attachement? (chapter 5) Briefly describe each one.

A: XXX

- Q: First create a graph consisting of a single link. (You can call the nodes anything, but I would simply use integers as names).

- Q: Now add another node, connecting one of the existing nodes in proportion to their degree.

- Q: Keep going until you have a 100 node network.
Hint: The difficult part here is connecting to each node according to their degree. The way I do it is: generate a list of all edges (e.g. pairs of nodes), then flatten it (e.g. remove connection information). That list contains each node in proportion to its connections, thus drawing a random node from that list (e.g. using random.choice) corresponds to selecting a node with probability proportional to it's degree.

In [10]:
### Insert code here

- Q: Plot the network.

In [11]:
### Insert code here

- Q: Add more nodes until you have a 5000 node network.
- Q: What's the maximum and minimum degree?
- Q: Now, bin the degree distribution using numpy.histogram.
- Q: Plot the distribution. Plot it with both linear and log-log axes.

### Assignment 1.2: Stats and visualization of the Rock Music Network

#### Explain your process in words

- Q: Document how you crawled the network.
First, simply describe the steps of the process (what you did, step by step)
Then, write a short section about which part of the process that was most difficult
Next write a short section about how you used LLMs to help you
Finally, compose a short section about what you would do differently if you had to do it again

#### Simple network statistics and analysis

In [12]:
# DOWNLOAD THE DATA
filename = "assignment_1_rock_bands.pkl"

url = "https://raw.githubusercontent.com/sorenstange/02805_Social_graphs_and_interactions/main/assignment_1_rock_bands.pkl"

# Step 1: Check if file exists
if not os.path.exists(filename):
    # Download the file if it does not exist
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print("File downloaded successfully.")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
else:
    print("File already exists. Skipping download.")

# Step 2: Load the .pickle file
with open(filename, "rb") as f:
    data = pickle.load(f)

File already exists. Skipping download.


In [20]:
def clean_links(links):
    arr = []
    for l in links:
        match = re.search(r'|', l)
        if match:
            splits = l.split(r'|')
            arr.append(splits[0].replace(' ', '_'))
        else:
            arr.append(l.replace(' ', '_'))
    return arr

def analyze_node(data_point):
    Node = data_point['page_name']
    text = data_point['content']
    words = len(text.split())
    links_to = re.findall(r'\[\[([^\]]+)\]\]', text)
    links_to = clean_links(links_to)
    return Node, links_to, words

def create_network(data):
    G = nx.DiGraph()
    Nodes = list(set([data_point['page_name'] for data_point in data]))
    for data_point in data:
        Node, links_to, words = analyze_node(data_point)
        G.add_node(Node, words=words)
        for links_to_node in links_to:
            if links_to_node in Nodes:
                G.add_edge(Node, links_to_node)
    return G

G = create_network(data)

In [21]:
print('Q: What is the number of nodes in the network?')
print(f'A: There are {len(G.nodes())} nodes in the network.\n')
print('Q: What is the number of links in the network?')
print(f'A: There are {len(G.edges())} links in the network.\n')

Q: What is the number of nodes in the network?
A: There are 489 nodes in the network.

Q: What is the number of links in the network?
A: There are 7678 links in the network.



- Q: What is the number of nodes in the network?
- Q: More importantly, what is the number of links? (Chat with a fellow student or a TA to make sure you're in the right ball-park)
- Q: Plot the in and out-degree distributions for the entire network. What do you observe? Can you explain why the in-degree distribution is different from the out-degree distribution?
- Q: Compare the out-degree distribution to a random network with the same number of nodes and links.
- Q: Compare the in-degree distribution to a scale-free network with the same number of nodes.
- Q: Who are the top 5 most connected performers (Report results for in-degrees and out-degrees, that is, who has highest in-degree, who has highest out-degree)? Comment on your findings. Is this what you would have expected?
- Q: What are the 10 pages with the longest wiki entries? (use the length of content attribute to figure this one out)?

#### Let's build a simple visualization of the network

1. For the sake of the visualisation, let's convert our network to undirected graph (tip: There is a NetworkX command to help you).
- Note: Keep the directed graph, we will use it in the following exercises.
2. Use the NetworkX command nx.spring_layout or nx.draw_kamada_kawai to draw the resulting undirected network. (You can find background on the algorithms here.)
- Set up your plot so that node-size depends on the node degree.
- Make the node color depend on the length of content attribute. I recommend choosing a color scheme that is quite simple (e.g. the Sequential ones here: https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [15]:
### INSERT CODE HERE